https://python.langchain.com/docs/use_cases/chatbots/quickstart

The chatbot interface is based around messages rather than raw text, and is best suited to Chat Models rather than text completion LLMs.

Generally the difference between Models and ChatModels, PromptTemplates and ChatPromptTemplates, Prompts and Messages, is that the formers are designed for string input->outputs, whereas the latters are designed for lists of conversations.

In [ ]:
%pip install --upgrade langchain langchain-openai typing-inspect typing_extensions pydantic

## A Basic ChatBot

In [5]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.2)

In [3]:
chat_model.invoke("Translate this sentence from English to French: I love programming.")

AIMessage(content="J'adore la programmation.")

This input prompt can be wrapped more properly to differentiated better from AI messages:

In [6]:
from langchain_core.messages import HumanMessage
chat_model.invoke([HumanMessage("Translate this sentence from English to French: I love programming.")])

AIMessage(content="J'adore la programmation.")

The model on its own does not have any history or state, so if you ask:

In [7]:
chat_model.invoke([HumanMessage(content="What did you just say?")])

AIMessage(content='I said, "What did you just say?"')

It doesn’t take the previous conversation turn into context, and cannot answer the question.

To get around this, **we need to pass the entire conversation history** into the model:

In [6]:
from langchain_core.messages import AIMessage

In [9]:
chat_model.invoke(
    [
        HumanMessage("Translate this sentence from English to French: I love programming."),
        AIMessage(content="J'adore la programmation."),
        HumanMessage("What did you just say?")
    ]
)

AIMessage(content='I said "J\'adore la programmation" which means "I love programming" in French.')

In [12]:
chat_model.invoke(
    [
        HumanMessage("Translate this sentence from English to French: I love programming."),
        AIMessage("J'adore la programmation."),
        HumanMessage("What did you just say?")
    ]
)

AIMessage(content='I said "J\'adore la programmation," which means "I love programming" in French.')

### Prompt Templates

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | chat_model

In the above definition of `prompt`, `MessagesPlaceholder(variable_name="messages")` is going to be responsible for taking in the entire set of input messages all at once.

In [15]:
chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate this sentence from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
)

AIMessage(content='I said "J\'adore la programmation," which means "I love programming" in French.')

### Message History

We can use an **in-memory**, demo message history called `ChatMessageHistory` for convenience.

In [1]:
from langchain.memory import ChatMessageHistory

In [2]:
chat_history = ChatMessageHistory()

chat_history.add_user_message("hi!")

chat_history.add_ai_message("whats up?")

chat_history.messages

[HumanMessage(content='hi!'), AIMessage(content='whats up?')]

In [8]:
chat_history.add_user_message(
    "Translate this sentence from English to French: I love programming."
)

response = chain.invoke({"messages": chat_history.messages})

response

AIMessage(content='The translation of "I love programming" in French is "J\'adore la programmation."')

In [9]:
chat_history.add_ai_message(response)

chat_history.add_user_message("What did you just say?")

chain.invoke({"messages": chat_history.messages})

AIMessage(content='I said, "J\'adore la programmation," which means "I love programming" in French.')

In [10]:
chat_history.messages

[HumanMessage(content='hi!'),
 AIMessage(content='whats up?'),
 HumanMessage(content='Translate this sentence from English to French: I love programming.'),
 HumanMessage(content='Translate this sentence from English to French: I love programming.'),
 AIMessage(content='The translation of "I love programming" in French is "J\'adore la programmation."'),
 HumanMessage(content='What did you just say?')]

This is a basic chatbot!!!

## Retrievers

In order to pull domain-specific knowledge for our chatbot, we can use `retrievers`.

The dependencies needed for retriever:

In [11]:
%pip install --upgrade --quiet chromadb beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


### Web-doc retrieval

We use the `LangSmith documentation` as source and store it in a `Chromadb` vectorstore for retrieval.

A WebBaseLoader from document_loaders can be used for loading web pages:

In [12]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
data = loader.load()

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Defining a text_splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# Applying the text_splitter to our web data
all_splits = text_splitter.split_documents(data)

Then we embed and store those chunks in a vector database:

In [14]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# This line grabs the document chunks, takes them to the embedding space, and stores them in the db.
vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

Now, we can test our retriever:

In [17]:
# k is the number of items to retrieve
retriever = vectorstore.as_retriever(k=4)

docs = retriever.invoke("Is Langsmith free?")

docs

[Document(page_content='LangSmith | 🦜️🛠️ LangSmith', metadata={'description': 'Introduction', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith | 🦜️🛠️ LangSmith'}),
 Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework', metadata={'description': 'Introduction', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith | 🦜️🛠️ LangSmith'}),
 Document(page_content='for building with LLMs.LangSmith is developed by LangChain, the company behind the 

### Represent Documents as Context

The `create_stuff_documents_chain` helper function packs all of the input documents into the prompt, as the context.

Other arguments (like messages) will be passed directly through into the prompt:

In [21]:
from langchain.chains.combine_documents import create_stuff_documents_chain

chat_model = ChatOpenAI(model="gpt-3.5-turbo-1106")

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat_model, prompt)

In [24]:
from langchain.memory import ChatMessageHistory

chat_history = ChatMessageHistory()

chat_history.add_user_message("how can langsmith help with testing?")

document_chain.invoke(
    {
        "messages": chat_history.messages,
        "context": docs,
    }
)

'LangSmith can help with testing by providing capabilities to debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework. It seamlessly integrates with LangChain, the open source framework for building with LLMs. Through LangSmith, users can access tracing and evaluation quick starts to begin testing their applications, and they can also learn about the workflows and capabilities supported at each stage of the LLM application lifecycle through the User Guide. Additionally, LangSmith offers a Prompt Hub, a prompt management tool built into the platform, to further aid in testing and development.'

We have an answer, generated from the relevant retrieved information of the Langsmith.

### Creating a retrieval chain